In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  x = df[ feats ].values
  y = df[ 'prices_amountmin' ].values

  scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df[ 'brand_cat' ] = df[ 'brand' ].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model([ 'brand_cat' ], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [0]:
df['features'].head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_shoe width', 'feat_frame depth', 'feat_label', 'feat_age end',
       'feat_contains cubic zirconia', 'feat_has expiration',
       'feat_polarized?', 'feat_sleeve length', 'feat_motion control',
       'feat_smart watch'],
      dtype='object', length=526)

In [0]:
df[ df['feat_athlete'].isnull() ].shape

(18272, 526)

In [0]:
df[ False == df['feat_athlete'].isnull() ].shape[0]/ df.shape[0] * 100

0.0437636761487965

In [0]:
df.shape[0]

18280

In [0]:
keys_stats = {}

for key in keys:
  keys_stats[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0]/ df.shape[0] * 100

In [0]:
{k:v for k,v in keys_stats.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df[ 'feat_brand_cat' ] = df[ 'feat_brand' ].factorize()[0]
df[ 'feat_color_cat' ] = df[ 'feat_color' ].factorize()[0]
df[ 'feat_gender_cat' ] = df[ 'feat_gender' ].factorize()[0]
df[ 'feat_manufacturer part number_cat' ] = df[ 'feat_manufacturer part number' ].factorize()[0]
df[ 'feat_material_cat' ] = df[ 'feat_material' ].factorize()[0]

df[ 'feat_sport_cat' ] = df[ 'feat_sport' ].factorize()[0]
df[ 'feat_style_cat' ] = df[ 'feat_style' ].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
df['brand'] = df['brand'].map( lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [88]:
x = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x, y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(x, y)
eli5.show_weights(perm, feature_names=feats)

(-57.16930822738809, 4.299656522467505)


Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [78]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.07714370283024, 4.249133563957477)

In [82]:
feats_cat = [x for x in df.columns if 'cat' in x]
featsfeats_cat

['categories',
 'brand_cat',
 'feat_location - country',
 'feat_location - city/state',
 'feat_shoe category',
 'feat_certifications and listings',
 'feat_catalog',
 'feat_recommended location',
 'feat_fabrication',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_primary shelf id_cat',
 'feat_hat style_cat',
 'feat_country of manufacturer_cat',
 'feat_color family_cat',
 'feat_lens width_cat',
 'feat_number of pieces_cat',
 'feat_capacity_cat',
 'feat_picture_cat',
 'feat_gps_cat',
 'feat_casual & dress shoe style_cat',
 'feat_safety feature_cat',
 'feat_fabric care instructions_cat',
 'feat_type 2_cat',
 'feat_construction_cat',
 'feat_safety features_cat',
 'feat_age group_cat',
 'feat_full product manual_cat',
 'feat_leather grade_cat',
 'feat_applicable_cat',
 'feat_made in_cat',
 'feat_band width_cat',
 'feat_in

In [0]:
feats = ['brand_cat', 'feat_brand_cat','feat_metal type_cat', 'feat_shape_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [90]:
x = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x, y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(x, y)
eli5.show_weights(perm, feature_names=feats)

(-57.25613237390231, 4.175607583121369)


Weight,Feature
0.2562 ± 0.0077,brand_cat
0.1035 ± 0.0075,feat_material_cat
0.0253 ± 0.0026,feat_gender_cat
0.0178 ± 0.0008,feat_brand_cat
0.0126 ± 0.0006,feat_shape_cat
0.0089 ± 0.0012,feat_metal type_cat
0.0031 ± 0.0011,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [74]:
df['brand'].value_counts(normalize=True)

nike              0.097210
puma              0.033315
ralph lauren      0.028775
vans              0.021116
new balance       0.020295
                    ...   
urban pipeline    0.000055
w.k               0.000055
rance             0.000055
wood n stream     0.000055
pleaser shoes     0.000055
Name: brand, Length: 1732, dtype: float64

In [75]:
df[ df['brand'] == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [80]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'style': 'running, cross training', 'gender': 'mens', 'color': 'white/metallic gold-midnight navy'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/volt/dark grey', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '700155070', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'condition': 'new with box', 'type': 'cleats'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'black/white/dark grey/pure platinum', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '620465011', 'brand': 'nike', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'}],
      dtype=object)

In [94]:
ls

data/  HelloGithub.ipynb  matrix_one/


In [92]:
!git add day5.ipynb

fatal: pathspec 'day5.ipynb' did not match any files
